In [17]:
import torch
import json
import transformer as t
import importlib
importlib.reload(t);

In [41]:
#
# Hyper parameter
#
topic           = 'commits'
token_size      = 3
batch_size      = 128
max_iterations  = 11000
eval_interval   = 2000
learning_rate   = 1e-4
eval_iters      = 200

#
# Network 
#
t.attention_heads_per_block = 8
t.attention_blocks          = 16
t.sample_size               = 48     # number of consecutive characters to predict from
t.embedding_size            = 256    # size of the embedding vectors
t.dropout                   = 0.2

# remember to set t.vocabulary_size when data is loaded

In [27]:
# read in file
with open(f'{topic}/{topic}.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text = text.ljust((len(text) + token_size-1)//token_size*token_size, '\n')

tokens = list(set([text[i:i+token_size] for i in range(0, len(text), token_size)]))
t.vocabulary_size = len(tokens)

# Build encoder and decoder
token_to_index = {token:index for index,token in enumerate(tokens)}
index_to_token = {index:token for index,token in enumerate(tokens)}
def encode(s):
    s = s.ljust((len(s) + token_size-1)//token_size * token_size, '\n')
    return [token_to_index[token] for token in [s[i:i+token_size] for i in range(0, len(s), token_size)]]
encode = lambda s: [token_to_index[token] for token in [s[i:i+token_size] for i in range(0, len(s), token_size)]]
decode = lambda l: ''.join([index_to_token[i] for i in l])

# save vocabulary
dictionary = {str(k): v for k, v in index_to_token.items()}
with open(f'{topic}/vocabulary.json', 'w') as file:
    json.dump(dictionary, file)

# create data tensors
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
training_data = data[:n]
validation_data = data[n:]

In [ ]:
#
# Create model
#
torch.manual_seed(1337)
model = t.Transformer()
m = model.to(t.device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
print(f'{sum(p.nelement() for p in model.parameters())} parameters')

current_iteration = 10000

# Load the states
checkpoint = torch.load(f'{topic}/{topic}-{current_iteration}.nn');
model.load_state_dict(checkpoint['model_state_dict']);
optimizer.load_state_dict(checkpoint['optimizer_state_dict']);

model.train()

In [42]:
#
# mini-batch creation
#
def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - t.sample_size, (batch_size, ))
    x = torch.stack([data[i:i+t.sample_size] for i in ix])
    y = torch.stack([data[i+1:i+t.sample_size+1] for i in ix])
    x,y = x.to(t.device), y.to(t.device)
    return x,y
#
# evaluation
#
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

def save_model(step):
    model.eval()
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'{topic}/{topic}-{step}.nn');
    model.train()

def export_model(step):
    model.eval()
    dummy_input = torch.randint(low=0, high=t.vocabulary_size, size=(1, t.sample_size), dtype=torch.long)
    torch.onnx.export(model, dummy_input, f"{topic}/{topic}-{step}.onnx")
    model.train()    

In [43]:
#
# Training
#
model.train()
for _ in range(max_iterations-current_iteration):
    current_iteration += 1
    if current_iteration % eval_interval == 0 and current_iteration > 10000:
        print("evaluating...")
        losses = estimate_loss()
        print(f"step {current_iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    if current_iteration % 500 == 0:
        print(f"step {current_iteration}")

    if current_iteration % 1000 == 0 and current_iteration > 10000:
        print(f"Saving at step {current_iteration}")
        save_model(current_iteration)

    xb, yb = get_batch('train')
    _, loss = model(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

save_model(current_iteration)
export_model(current_iteration)

step 10500


In [ ]:
model.eval()
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=t.device), max_tokens=500)[0].tolist()))